# Computational Theory Assessment

## Problem 1: Binary Words and Operations

### SHA-1 Parity Function

The Parity function is one of four logical functions used in the SHA-1 hash algorithm, as specified in [FIPS 180-4 Section 4.1.1](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf). This function operates on three 32-bit words and produces a single 32-bit output through bitwise exclusive OR (XOR) operations.

In SHA-1, different logical functions are applied during different rounds of the compression function. The Parity function is specifically used during rounds 0-19 and 60-79 (designated as rounds where `t` satisfies `0 ≤ t ≤ 19` or `60 ≤ t ≤ 79`).

**Mathematical Definition:**

```
Parity(x, y, z) = x ⊕ y ⊕ z
```

Where `⊕` represents the bitwise XOR operation.

**Properties:**

The function returns `1` at each bit position where an odd number of the corresponding input bits are `1`, and returns `0` where an even number (including zero) of input bits are `1`. This behavior is characteristic of XOR operations and gives the function its "parity" designation.

**Key Characteristics:**
- **Commutative**: The order of operands does not affect the result
- **Associative**: Grouping of operations does not affect the result
- **Self-inverse**: XORing a value twice with the same operand returns the original value

**Example (4-bit):**

Using the values from our test: `Parity(0b1010, 0b1100, 0b0011) = 0b0101`

| Bit Position | x | y | z | x⊕y⊕z |
|--------------|---|---|---|-------|
| 3 (MSB)      | 1 | 1 | 0 | 0     |
| 2            | 0 | 1 | 0 | 1     |
| 1            | 1 | 0 | 1 | 0     |
| 0 (LSB)      | 0 | 0 | 1 | 1     |

Result: `1010 ⊕ 1100 ⊕ 0011 = 0101`


**References:**
- [FIPS 180-4: Secure Hash Standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) - Section 4.1.1

In [1]:
import numpy as np

def Parity(x, y, z):
    """
    SHA-1 Parity function: x ^ y ^ z.
    
    Computes the bitwise XOR of three 32-bit words.
    This function is used in SHA-1 and returns 1 for each bit position
    where an odd number of the corresponding bits in x, y, z are 1.
    
    Args:
        x: 32-bit unsigned integer
        y: 32-bit unsigned integer
        z: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of x XOR y XOR z
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    result = x ^ y ^ z

    return np.uint32(result)

In [2]:
def test_parity():
    """Test the Parity function with various inputs."""
    
    # Test 1: Identity - all zeros
    assert Parity(0x00000000, 0x00000000, 0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: Single bit set
    assert Parity(0x00000001, 0x00000000, 0x00000000) == 0x00000001, \
        "Single 1 bit should return 1"
    
    # Test 3 All ones
    assert Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 4: Mixed bits test
    assert Parity(0x00000005, 0x00000003, 0x00000001) == 0x00000007, \
        "Mixed bits: 0101 XOR 0011 XOR 0001 = 0111"
    
    # Test 5: 4-bit test
    assert Parity(0b1010, 0b1100, 0b0011) == 0b0101, \
        "Binary: 1010 XOR 1100 XOR 0011 = 0101"
    print("All Parity tests passed")

# Run the tests
test_parity()

All Parity tests passed


In [3]:
def Ch(x, y, z):
    """
    SHA-1 Ch (Choose) function: (x & y) ^ (~x & z).
    
    The Ch function "chooses" between y and z based on the bits of x:
    - If a bit in x is 1, the corresponding bit from y is chosen
    - If a bit in x is 0, the corresponding bit from z is chosen
    
    Args:
        x: 32-bit unsigned integer (selector)
        y: 32-bit unsigned integer (chosen when x bit is 1)
        z: 32-bit unsigned integer (chosen when x bit is 0)
    
    Returns:
        32-bit unsigned integer result of (x & y) ^ (~x & z)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # (x AND y) XOR (NOT x AND z)
    result = (x & y) ^ (~x & z)
    
    return np.uint32(result)

In [4]:
def test_ch():
    """Test the Ch function with various inputs."""
    
    # Test 1: All ones in x (choose all from y)
    assert Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF00) == 0x12345678, \
        "When x is all 1s, result should equal y"
    
    # Test 2: All zeros in x (choose all from z)
    assert Ch(0x00000000, 0x12345678, 0xABCDEF00) == 0xABCDEF00, \
        "When x is all 0s, result should equal z"
    
    # Test 3: Alternating pattern
    assert Ch(0xAAAAAAAA, 0xFFFFFFFF, 0x00000000) == 0xAAAAAAAA, \
        "With alternating x bits, should select alternating bits from y and z"
    
    # Test 4: 4-bit test
    assert Ch(0b1100, 0b1010, 0b0101) == 0b1001, \
        "Binary: Ch(1100, 1010, 0101) should equal 1001"
    
    # Test 5: Actual hash values
    expected5 = np.uint32((0x67452301 & 0xEFCDAB89) ^ (~np.uint32(0x67452301) & 0x98BADCFE))
    assert Ch(0x67452301, 0xEFCDAB89, 0x98BADCFE) == expected5, \
        "Calculation with SHA-1 constants should match direct implementation"
    
    print("All Ch tests passed")

# Run the tests
test_ch()

All Ch tests passed


In [5]:
def Maj(x, y, z):
    """
    SHA-1 Maj (Majority) function: (x & y) ^ (x & z) ^ (y & z).
    
    The Maj function returns the majority vote for each bit position:
    - Returns 1 if at least 2 of the 3 corresponding bits are 1
    - Returns 0 if at least 2 of the 3 corresponding bits are 0
    
    Args:
        x: 32-bit unsigned integer
        y: 32-bit unsigned integer
        z: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of (x & y) ^ (x & z) ^ (y & z)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # (x AND y) XOR (x AND z) XOR (y AND z)
    result = (x & y) ^ (x & z) ^ (y & z)
    
    return np.uint32(result)

In [6]:
def test_Maj():
    """Test the Maj function with various inputs."""
    
    # Test 1: All zeros
    assert Maj(0x00000000, 0x00000000, 0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: All ones
    assert Maj(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 3: Two ones, one zero (majority = 1)
    assert Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000) == 0xFFFFFFFF, \
        "Two 1s, one 0 should return 1 for each bit"
    
    # Test 4: Two zeros, one one (majority = 0)
    assert Maj(0x00000000, 0x00000000, 0xFFFFFFFF) == 0x00000000, \
        "Two 0s, one 1 should return 0 for each bit"
    
    # Test 5: 4-bit test with mixed pattern
    assert Maj(0b1100, 0b1010, 0b1001) == 0b1000, \
        "Binary: Maj(1100, 1010, 1001) should equal 1000"

    print("All Maj tests passed")

# Run the tests
test_Maj()

All Maj tests passed


In [7]:
def Sigma0(x):
    """
    SHA-256 Sigma0 (Σ₀²⁵⁶) function: ROTR²(x) ^ ROTR¹³(x) ^ ROTR²²(x)
    
    This function performs three right rotations on a 32-bit word and XORs them.
    
    ROTR^n(x) = circular right rotation of x by n positions
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR²(x) ^ ROTR¹³(x) ^ ROTR²²(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 2, 13, and 22 bits
    rotr2 = np.uint32((x >> 2) | (x << 30))   # ROTR²(x)
    rotr13 = np.uint32((x >> 13) | (x << 19)) # ROTR¹³(x)
    rotr22 = np.uint32((x >> 22) | (x << 10)) # ROTR²²(x)
    
    # XOR all three rotations
    result = rotr2 ^ rotr13 ^ rotr22
    
    return np.uint32(result)

In [8]:
def test_Sigma0():
    """Test the Sigma0 function with various inputs."""
    
    # Test 1: All zeros
    assert Sigma0(0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: All ones
    assert Sigma0(0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 3: Single bit set
    x = 0x00000001
    rotr2 = np.uint32((x >> 2) | (x << 30))   # 0x40000000
    rotr13 = np.uint32((x >> 13) | (x << 19)) # 0x00020000
    rotr22 = np.uint32((x >> 22) | (x << 10)) # 0x00000400
    expected = rotr2 ^ rotr13 ^ rotr22        # 0x40020400
    assert Sigma0(0x00000001) == expected, \
        f"Single bit rotation failed, expected 0x{expected:08x}"
    
    # Test 4: Alternating bit pattern
    assert Sigma0(0xAAAAAAAA) != 0xAAAAAAAA, \
        "Rotation of alternating pattern should change the pattern"
    
    print("All Sigma0 tests passed")

# Run the tests
test_Sigma0()

All Sigma0 tests passed


In [9]:
def Sigma1(x):
    """
    SHA-256 Sigma1 (Σ₁²⁵⁶) function: ROTR⁶(x) ^ ROTR¹¹(x) ^ ROTR²⁵(x)
    
    This function performs three right rotations on a 32-bit word and XORs them.
    
    ROTR^n(x) = circular right rotation of x by n positions
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR⁶(x) ^ ROTR¹¹(x) ^ ROTR²⁵(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 6, 11, and 25 bits
    rotr6 = np.uint32((x >> 6) | (x << 26))   # ROTR⁶(x)
    rotr11 = np.uint32((x >> 11) | (x << 21)) # ROTR¹¹(x)
    rotr25 = np.uint32((x >> 25) | (x << 7))  # ROTR²⁵(x)
    
    # XOR all three rotations
    result = rotr6 ^ rotr11 ^ rotr25
    
    return np.uint32(result)

In [10]:
def test_Sigma1():
    """Test the Sigma1 function with various inputs."""
    
    # Test 1: All zeros
    assert Sigma1(0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: All ones
    assert Sigma1(0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 3: Single bit set
    x = 0x00000001
    rotr6 = np.uint32((x >> 6) | (x << 26))   # 0x04000000
    rotr11 = np.uint32((x >> 11) | (x << 21)) # 0x00200000
    rotr25 = np.uint32((x >> 25) | (x << 7))  # 0x00000080
    expected = rotr6 ^ rotr11 ^ rotr25        # 0x04200080
    assert Sigma1(0x00000001) == expected, \
        f"Single bit rotation failed, expected 0x{expected:08x}"
    
    print("All Sigma1 tests passed")

# Run the tests
test_Sigma1()

All Sigma1 tests passed


In [11]:
def sigma0(x):
    """
    SHA-256 sigma0 (σ₀²⁵⁶) function: ROTR⁷(x) ^ ROTR¹⁸(x) ^ SHR³(x)
    
    This function performs two right rotations and one right shift on a 32-bit word and XORs them.
    Used in the message schedule of SHA-256.
    
    ROTR^n(x) = circular right rotation of x by n positions
    SHR^n(x) = right shift of x by n positions (no wrap around)
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR⁷(x) ^ ROTR¹⁸(x) ^ SHR³(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 7 and 18 bits
    rotr7 = np.uint32((x >> 7) | (x << 25))   # ROTR⁷(x)
    rotr18 = np.uint32((x >> 18) | (x << 14)) # ROTR¹⁸(x)
    
    # Right shift by 3 bits (no wrap around)
    shr3 = np.uint32(x >> 3)                  # SHR³(x)
    
    # XOR all three operations
    result = rotr7 ^ rotr18 ^ shr3
    
    return np.uint32(result)

In [12]:
def test_Sigma0():
    """Test the SHA-256 Sigma0 function with various inputs."""
    
    # Test 1: All zeros
    assert Sigma0(0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: All ones
    assert Sigma0(0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 3: Single bit test
    x = 0x00000001
    rotr2 = np.uint32((x >> 2) | (x << 30))   # 0x40000000
    rotr13 = np.uint32((x >> 13) | (x << 19)) # 0x00080000
    rotr22 = np.uint32((x >> 22) | (x << 10)) # 0x00000400
    expected = rotr2 ^ rotr13 ^ rotr22        # 0x40080400
    assert Sigma0(0x00000001) == expected, \
        f"Single bit rotation failed, expected 0x{expected:08x}"
    
    print("All Sigma0 tests passed")

# Run the tests
test_Sigma0()

All Sigma0 tests passed


In [13]:
def sigma1(x):
    """
    SHA-256 sigma1 (σ₁²⁵⁶) function: ROTR¹⁷(x) ^ ROTR¹⁹(x) ^ SHR¹⁰(x)
    
    This function performs two right rotations and one right shift on a 32-bit word and XORs them.
    Used in the message schedule of SHA-256 (extending the message).
    
    ROTR^n(x) = circular right rotation of x by n positions
    SHR^n(x) = right shift of x by n positions (no wrap around, fills with zeros)
    
    Reference: FIPS 180-4, Section 4.1.2 (SHA-256 Functions)
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        32-bit unsigned integer result of ROTR¹⁷(x) ^ ROTR¹⁹(x) ^ SHR¹⁰(x)
    """
    x = np.uint32(x)
    
    # Right rotate by 17 and 19 bits
    rotr17 = np.uint32((x >> 17) | (x << 15))  # ROTR¹⁷(x)
    rotr19 = np.uint32((x >> 19) | (x << 13))  # ROTR¹⁹(x)
    
    # Right shift by 10 bits (no wrap around)
    shr10 = np.uint32(x >> 10)                 # SHR¹⁰(x)
    
    # XOR all three operations
    result = rotr17 ^ rotr19 ^ shr10
    
    return np.uint32(result)

In [14]:
def test_Sigma1():
    """Test the SHA-256 Sigma1 function with various inputs."""
    
    # Test 1: All zeros
    assert Sigma1(0x00000000) == 0x00000000, \
        "All zeros should return 0"
    
    # Test 2: All ones
    assert Sigma1(0xFFFFFFFF) == 0xFFFFFFFF, \
        "All ones should return all ones"
    
    # Test 3: Single bit set
    x = 0x00000001
    rotr6 = np.uint32((x >> 6) | (x << 26))   # 0x04000000
    rotr11 = np.uint32((x >> 11) | (x << 21)) # 0x00200000
    rotr25 = np.uint32((x >> 25) | (x << 7))  # 0x00000080
    expected = rotr6 ^ rotr11 ^ rotr25        # 0x04200080
    assert Sigma1(0x00000001) == expected, \
        f"Single bit rotation failed, expected 0x{expected:08x}"

    print("All Sigma1 tests passed")

# Run the tests
test_Sigma1()

All Sigma1 tests passed


## Problem 2: Fractional Parts of Cube Roots


In [15]:
import numpy as np

def primes(n):
    """
    Generate the first n prime numbers.
    
    Reference: https://www.geeksforgeeks.org/sieve-of-eratosthenes/
    
    Args:
        n: Number of primes to generate
    
    Returns:
        List of first n prime numbers
    """
    if n <= 0:
        return []
    
    primes_list = []
    candidate = 2
    
    while len(primes_list) < n:
        is_prime = True
        
        # Check if candidate is divisible by any previously found prime
        for prime in primes_list:
            if prime * prime > candidate:
                break
            if candidate % prime == 0:
                is_prime = False
                break
        
        if is_prime:
            primes_list.append(candidate)
        
        candidate += 1
    
    return primes_list

In [16]:
def fractional_part_cube_root(prime):
    """
    Calculate the first 32 bits of the fractional part of the cube root of a prime.

    Reference: FIPS 180-4, Section 4.2.2
    https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
    
    The fractional part is obtained by:
    1. Computing the cube root
    2. Subtracting the integer part
    3. Scaling by 2^32 to get the first 32 bits
    
    Args:
        prime: Prime number
    
    Returns:
        32-bit unsigned integer representing the fractional part
    """
    # Calculate cube root
    cube_root = np.cbrt(prime)
    
    # Extract fractional part
    fractional = cube_root - np.floor(cube_root)
    
    # Scale to 32 bits: multiply by 2^32
    scaled = fractional * (2**32)
    
    # Convert to 32-bit unsigned integer
    result = np.uint32(scaled)
    
    return result


In [17]:
def generate_k_constants(n=64):
    """
    Generate SHA-256 K constants from cube roots of first n primes.
    
    These constants are used in the SHA-256 compression function.
    Reference: FIPS 180-4, Section 4.2.2, Page 11
    
    Args:
        n: Number of constants to generate (default: 64 for SHA-256)
    
    Returns:
        tuple: (list of K constants, list of primes used)
    """
    # Get first n primes
    prime_list = primes(n)
    
    # Calculate K constant from each prime's cube root
    k_values = []
    for p in prime_list:
        k = fractional_part_cube_root(p)
        k_values.append(k)
    
    return k_values, prime_list

In [18]:
def display_hex_result():
    """
     Display the 64 SHA-256 K constants in hexadecimal format.
    
    Prints 8 constants per line for easy comparison with FIPS 180-4.
    """
    k_vals, _ = generate_k_constants(64)
    
    for i in range(0, 64, 8):
        print([f"0x{k:08x}" for k in k_vals[i:i+8]])

display_hex_result()

['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5']
['0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', '0x9bdc06a7', '0xc19bf174']
['0xe49b69c1', '0xefbe4786', '0x0fc19dc6', '0x240ca1cc', '0x2de92c6f', '0x4a7484aa', '0x5cb0a9dc', '0x76f988da']
['0x983e5152', '0xa831c66d', '0xb00327c8', '0xbf597fc7', '0xc6e00bf3', '0xd5a79147', '0x06ca6351', '0x14292967']
['0x27b70a85', '0x2e1b2138', '0x4d2c6dfc', '0x53380d13', '0x650a7354', '0x766a0abb', '0x81c2c92e', '0x92722c85']
['0xa2bfe8a1', '0xa81a664b', '0xc24b8b70', '0xc76c51a3', '0xd192e819', '0xd6990624', '0xf40e3585', '0x106aa070']
['0x19a4c116', '0x1e376c08', '0x2748774c', '0x34b0bcb5', '0x391c0cb3', '0x4ed8aa4a', '0x5b9cca4f', '0x682e6ff3']
['0x748f82ee', '0x78a5636f', '0x84c87814', '0x8cc70208', '0x90befffa', '0xa4506ceb', '0xbef9a3f7', '0xc67178f2']


In [19]:
def test_primes():
    """Test the primes function with various inputs."""
    
    # Test 1: First 10 primes
    assert primes(10) == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29], \
        "First 10 primes incorrect"
    
    # Test 2: First 5 primes
    assert primes(5) == [2, 3, 5, 7, 11], \
        "First 5 primes incorrect"
    
    # Test 3: Edge case - first prime only
    assert primes(1) == [2], \
        "First prime should be 2"
    
    # Test 4: Zero primes (edge case)
    assert primes(0) == [], \
        "Zero primes should return empty list"
    
    print("All primes tests passed")

# Run the tests
test_primes()

All primes tests passed


In [20]:
def test_fractional_part():
    """Test the fractional_part_cube_root function properties."""
    
    # Test 1: Result is numpy uint32 type
    result = fractional_part_cube_root(2)
    assert isinstance(result, np.uint32), \
        "Result should be numpy uint32 type"
    
    # Test 2: First prime (2) produces correct value
    assert fractional_part_cube_root(2) == 0x428a2f98, \
        "Cube root fractional part of 2 incorrect"
    
    # Test 3: Prime 311 (last in SHA-256) produces correct value
    assert fractional_part_cube_root(311) == 0xc67178f2, \
        "Cube root fractional part of 311 incorrect"
    
    # Test 4: Deterministic - same input gives same output
    assert fractional_part_cube_root(127) == fractional_part_cube_root(127), \
        "Function should be deterministic"
    
    # Test 5: Result fits in 32 bits
    result = fractional_part_cube_root(311)
    assert result <= 0xFFFFFFFF, \
        "Result should fit in 32 bits"
    
    print("All fractional_part_cube_root tests passed")

# Run the tests
test_fractional_part()

All fractional_part_cube_root tests passed


In [21]:
def test_k_constants():
    """Test all 64 K constants against SHA-256 standard."""
    
    # Official SHA-256 K constants from FIPS 180-4, Section 4.2.2
    OFFICIAL_K = [
        0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
        0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
        0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
        0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
        0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
        0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
        0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
        0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
        0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
        0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
        0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
        0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
        0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
        0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
        0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
        0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
    ]
    
    # Generate K constants
    k_vals, _ = generate_k_constants(64)
    
    # Test 1: Correct number of constants generated
    assert len(k_vals) == 64, \
        "Should generate exactly 64 constants"
    
    # Test 2: First constant matches
    assert k_vals[0] == OFFICIAL_K[0], \
        f"First constant should be 0x{OFFICIAL_K[0]:08x}"
    
    # Test 3: Last constant matches
    assert k_vals[63] == OFFICIAL_K[63], \
        f"Last constant should be 0x{OFFICIAL_K[63]:08x}"
    
    # Test 4: Middle constant matches
    assert k_vals[32] == OFFICIAL_K[32], \
        f"Middle constant (index 32) should be 0x{OFFICIAL_K[32]:08x}"
    
    # Test 5: All constants match
    for i, (gen, off) in enumerate(zip(k_vals, OFFICIAL_K)):
        assert gen == off, \
            f"Constant {i} mismatch: generated 0x{gen:08x}, expected 0x{off:08x}"
    
    print("All K constants tests passed")

# Run the tests
test_k_constants()

All K constants tests passed


## Problem 3: Padding

In [22]:
def block_parse(msg):
    """
    Generator that yields 512-bit blocks from a message with proper SHA-256 padding.
    
    Takes a message in bytes and yields it in 64-byte chunks. The last chunk
    (or two chunks) will have padding added according to the SHA-256 spec.
    
    Args:
        msg (bytes): The message to process
        
    Yields:
        bytes: 64-byte blocks, with padding on the final block(s)
    """
    # First, figure out how long the message is
    original_length_bits = len(msg) * 8
    
    # Yield any complete 64-byte blocks we have
    i = 0
    while i + 64 <= len(msg):
        yield msg[i:i + 64]
        i += 64
    
    # Now handle whatever's left over (less than 64 bytes)
    leftover = msg[i:]
    
    # Start padding: add our message, then the required 0x80 byte
    current = leftover + b'\x80'
    
    # Check if we can fit the length field (8 bytes) in this block
    # We need the total to be 64 bytes, with 8 reserved for length
    # So if current <= 56 bytes, we're good for one block
    if len(current) <= 56:
        # Pad with zeros up to byte 56
        current = current + b'\x00' * (56 - len(current))
        # Add the length in the last 8 bytes
        current = current + original_length_bits.to_bytes(8, 'big')
        yield current
    else:
        # Won't fit! Need two blocks
        # First block: fill the rest with zeros
        current = current + b'\x00' * (64 - len(current))
        yield current
        
        # Second block: 56 zero bytes, then the length
        second_block = b'\x00' * 56 + original_length_bits.to_bytes(8, 'big')
        yield second_block

In [23]:
def test_block_parse():
    """Test block_parse with different message sizes."""
    
    # Test 1: Empty message
    blocks = list(block_parse(b''))
    assert len(blocks) == 1, "Empty message should give 1 block"
    assert len(blocks[0]) == 64, "Block should be 64 bytes"
    assert blocks[0][0] == 0x80, "Should start with 0x80"
    assert int.from_bytes(blocks[0][-8:], 'big') == 0, "Length should be 0"
    
    # Test 2: "abc" - the standard example
    blocks = list(block_parse(b'abc'))
    assert len(blocks) == 1, "'abc' should give 1 block"
    assert blocks[0][:3] == b'abc', "Should start with 'abc'"
    assert blocks[0][3] == 0x80, "Padding at byte 3"
    assert int.from_bytes(blocks[0][-8:], 'big') == 24, "Length should be 24 bits"
    
    # Test 3: 55 bytes - max for one block
    msg = b'A' * 55
    blocks = list(block_parse(msg))
    assert len(blocks) == 1, "55 bytes should fit in 1 block"
    assert blocks[0][:55] == msg, "Should contain full message"
    assert int.from_bytes(blocks[0][-8:], 'big') == 440, "Length should be 440 bits"
    
    # Test 4: 56 bytes - needs 2 blocks
    msg = b'B' * 56
    blocks = list(block_parse(msg))
    assert len(blocks) == 2, "56 bytes needs 2 blocks"
    assert blocks[0][:56] == msg, "First block has message"
    assert int.from_bytes(blocks[1][-8:], 'big') == 448, "Length in second block"
    
    # Test 5: 64 bytes - full block
    msg = b'C' * 64
    blocks = list(block_parse(msg))
    assert len(blocks) == 2, "64 bytes needs 2 blocks"
    assert blocks[0] == msg, "First block is just the message"
    assert blocks[1][0] == 0x80, "Second block starts with padding"
    
    print("All block_parse tests passed")

test_block_parse()

All block_parse tests passed


## Problem 4: Hashes

In [ ]:
def sha256_hash(message):
    """
    Compute the SHA-256 hash of a message.
    
    Reference: FIPS 180-4, Section 6.2.2
    https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
    
    Args:
        message (bytes): The input message to hash
        
    Returns:
        str: The 256-bit SHA-256 hash as a 64-character hexadecimal string
    """
    # SHA-256 K constants (first 32 bits of cube roots of first 64 primes)
    # FIPS 180-4, Section 4.2.2
    K = [
        0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
        0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
        0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
        0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
        0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
        0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
        0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
        0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
    ]
    
    # Initial hash values (first 32 bits of square roots of first 8 primes)
    # FIPS 180-4, Section 5.3.3
    H = [
        0x6a09e667,
        0xbb67ae85,
        0x3c6ef372,
        0xa54ff53a,
        0x510e527f,
        0x9b05688c,
        0x1f83d9ab,
        0x5be0cd19
    ]
    
    # Get padded message blocks
    blocks = list(block_parse(message))
    
    # Process each 512-bit block
    for block in blocks:
        # Prepare message schedule W (64 words)
        W = []
        
        # First 16 words come directly from the block
        for i in range(16):
            word = int.from_bytes(block[i*4:(i+1)*4], byteorder='big')
            W.append(word)
        
        # Extend to 64 words
        for t in range(16, 64):
            s0 = sigma0(W[t-15])
            s1 = sigma1(W[t-2])
            W.append((W[t-16] + s0 + W[t-7] + s1) & 0xFFFFFFFF)
        
        # Initialize working variables with current hash values
        a, b, c, d, e, f, g, h = H
        
        # Main compression loop (64 rounds)
        for t in range(64):
            T1 = (h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t]) & 0xFFFFFFFF
            T2 = (Sigma0(a) + Maj(a, b, c)) & 0xFFFFFFFF
            
            h = g
            g = f
            f = e
            e = (d + T1) & 0xFFFFFFFF
            d = c
            c = b
            b = a
            a = (T1 + T2) & 0xFFFFFFFF
        
        # Compute intermediate hash value
        H[0] = (H[0] + a) & 0xFFFFFFFF
        H[1] = (H[1] + b) & 0xFFFFFFFF
        H[2] = (H[2] + c) & 0xFFFFFFFF
        H[3] = (H[3] + d) & 0xFFFFFFFF
        H[4] = (H[4] + e) & 0xFFFFFFFF
        H[5] = (H[5] + f) & 0xFFFFFFFF
        H[6] = (H[6] + g) & 0xFFFFFFFF
        H[7] = (H[7] + h) & 0xFFFFFFFF
    
    # Produce final hash value (concatenate all 8 words)
    hash_result = ''
    for h_value in H:
        hash_result += format(h_value, '08x')
    
    return hash_result

In [25]:
def test_sha256():
    """Test SHA-256 hash function with official test vectors."""
    
    # Test 1: Empty string
    result = sha256_hash(b'')
    expected = 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'
    assert result == expected, f"Empty string test failed\nGot:      {result}\nExpected: {expected}"
    
    # Test 2: "abc" (FIPS 180-4 Appendix B.1)
    result = sha256_hash(b'abc')
    expected = 'ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad'
    assert result == expected, f"'abc' test failed\nGot:      {result}\nExpected: {expected}"
    
    # Test 3: Longer message (FIPS 180-4 Appendix B.2)
    message = b'abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq'
    result = sha256_hash(message)
    expected = '248d6a61d20638b8e5c026930c3e6039a33ce45964ff2167f6ecedd419db06c1'
    assert result == expected, f"Long message test failed\nGot:      {result}\nExpected: {expected}"
    
    # Test 4: Single character
    result = sha256_hash(b'a')
    expected = 'ca978112ca1bbdcafac231b39a23dc4da786eff8147c4e72b9807785afee48bb'
    assert result == expected, f"Single 'a' test failed\nGot:      {result}\nExpected: {expected}"
    
    # Test 5: Numbers
    result = sha256_hash(b'123')
    expected = 'a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa07e998e86f7f7a27ae3'
    assert result == expected, f"'123' test failed\nGot:      {result}\nExpected: {expected}"
    
    print("All SHA-256 tests passed!")

# Run tests
test_sha256()

All SHA-256 tests passed!


/tmp/ipykernel_3848/133285630.py:57: RuntimeWarning: overflow encountered in scalar add
  W.append((W[t-16] + s0 + W[t-7] + s1) & 0xFFFFFFFF)
/tmp/ipykernel_3848/133285630.py:64: RuntimeWarning: overflow encountered in scalar add
  T1 = (h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t]) & 0xFFFFFFFF
/tmp/ipykernel_3848/133285630.py:65: RuntimeWarning: overflow encountered in scalar add
  T2 = (Sigma0(a) + Maj(a, b, c)) & 0xFFFFFFFF
/tmp/ipykernel_3848/133285630.py:70: RuntimeWarning: overflow encountered in scalar add
  e = (d + T1) & 0xFFFFFFFF
/tmp/ipykernel_3848/133285630.py:74: RuntimeWarning: overflow encountered in scalar add
  a = (T1 + T2) & 0xFFFFFFFF
/tmp/ipykernel_3848/133285630.py:78: RuntimeWarning: overflow encountered in scalar add
  H[1] = (H[1] + b) & 0xFFFFFFFF
/tmp/ipykernel_3848/133285630.py:80: RuntimeWarning: overflow encountered in scalar add
  H[3] = (H[3] + d) & 0xFFFFFFFF
/tmp/ipykernel_3848/133285630.py:81: RuntimeWarning: overflow encountered in scalar add
  H[4] =

## Problem 5: Passwords

# End